In [1]:
import tensorflow as tf

# A generator that provides a representative dataset
base_dir = '../cats_and_dogs_small/train'
IMAGE_SIZE = 150
def representative_data_gen():
  dataset_list = tf.data.Dataset.list_files( base_dir + '/*/*' )
  for i in range(100):
    image = next( iter( dataset_list ))
    image = tf.io.read_file( image )
    image = tf.io.decode_jpeg( image, channels=3 )
    image = tf.image.resize( image, [IMAGE_SIZE, IMAGE_SIZE] )
    image = tf.cast( image / 255., tf.float32 )
    image = tf.expand_dims( image, 0 )
    yield [image]

model = tf.keras.models.load_model( 'cats_and_dogs_small_1.h5' )
converter = tf.lite.TFLiteConverter.from_keras_model( model )

# This enables quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# This sets the representative dataset for quantization
converter.representative_dataset = representative_data_gen

# This ensures that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

# For full integer quantization, though supported types defaults to int8 only, we explicitly declare it for clarity.
converter.target_spec.supported_types = [tf.int8]

# These set the input and output tensors to uint8 (added in r2.3)
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
tflite_model = converter.convert()

with open('cats_and_dogs_quant.tflite', 'wb') as f:
  f.write(tflite_model)